# Теория вероятностей и математическая статистика

### Курсовая работа

Исследовать корреляционную зависимость и выявить методами парной регрессии зависимость для динамической вязкости нефтяной эмульсии от обводненности при различных плотностях нефти и различных значениях температуры и давления для различных степеней деспергирования нефти в воде или воды в нефти. Исследовать инверсию фаз.

Для исследования желательно следующие данные:
- вязкость эмульсии,
- обводненность эмульсии,
- плотность нефти,
- плотность воды,
- вязкость воды,
- вязкость чистой нефти,
- температура эмульсии,
- давление при котором происходить измерение,
- размер зерна несомой фазы (воды в нефти или нефти в воде).

Нашел экспериментальные данные близкие по постановке задачи, точных не нашел. В Шлюмберже пока не ответили.